In [ ]:
# this notebook is to test run the custom stock trading environment

In [1]:
import os
import re

# import libraries for training stable baselines agent
from train_stable_agent import create_stable_agents, evaluate_stable_agent, train_stable_agent, output_stable_agent, full_run

from stable_baselines3.common.env_checker import check_env

# import custom functions and classes
from curatedataset import makegymenv, run_env

from get_agent import Agent, TradingAlgorithm


In [2]:
full_run("train_config_TSLA.json")

[*********************100%%**********************]  1 of 1 completed


/usr/local/lib/python3.10/dist-packages/ta/utils.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(value)
/usr/local/lib/python3.10/dist-packages/ta/utils.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(value)
/usr/local/lib/python3.10/dist-packages/ta/utils.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(value)
/usr/local/lib/python3.10/dist-packages/ta/utils.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(val

init env with max step:  964
Vectorizing environment


/usr/local/lib/python3.10/dist-packages/ta/utils.py:33: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(method='bfill')
/usr/local/lib/python3.10/dist-packages/ta/utils.py:33: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(method='bfill')
/usr/local/lib/python3.10/dist-packages/ta/utils.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="ffill").fillna(value)
/usr/local/lib/python3.10/dist-packages/ta/utils.py:35: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series = series_output.fillna(method="

Creating PPO agent
Using cuda device
Creating A2C agent
Using cuda device
Creating DDPG agent
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Evaluate pre training model
Evaluating model:  <stable_baselines3.ppo.ppo.PPO object at 0x7f4eb4f7cac0>
mean_reward:201621.80 +/- 25780.68
Evaluating model:  <stable_baselines3.a2c.a2c.A2C object at 0x7f4eb455cdc0>
mean_reward:200618.71 +/- 8005.87
Evaluating model:  <stable_baselines3.ddpg.ddpg.DDPG object at 0x7f4eb455d030>
mean_reward:0.00 +/- 0.00
Training model:  <stable_baselines3.ppo.ppo.PPO object at 0x7f4eb4f7cac0>
------------------------------
| time/              |       |
|    fps             | 4106  |
|    iterations      | 1     |
|    time_elapsed    | 2     |
|    total_timesteps | 12288 |
------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 651         |
|    iterations           | 2           

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:278: UserWarning: Path 'trained_stable_agents' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


-------------------------------------
| time/                 |           |
|    fps                | 1150      |
|    iterations         | 100       |
|    time_elapsed       | 2         |
|    total_timesteps    | 3000      |
| train/                |           |
|    entropy_loss       | -2.8      |
|    explained_variance | -1.19e-07 |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | 3.17e+03  |
|    std                | 0.98      |
|    value_loss         | 1.26e+06  |
-------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 1139     |
|    iterations         | 200      |
|    time_elapsed       | 5        |
|    total_timesteps    | 6000     |
| train/                |          |
|    entropy_loss       | -2.78    |
|    explained_variance | 0.000171 |
|    learning_rate      | 0.0007   |
|    n_updates          | 199      |
|    policy_loss       

In [2]:
# make gym environment
stock_name = 'TSLA'
start_date = '2018-01-01'
num_days = 365*4
interval = '1d'
init_balance = 20000
norm_env, obs_space, act_space, col, data = makegymenv(stock_name, start_date, num_days, interval, normalize=True, init_balance=init_balance)
env,obs_space, act_space, col, data = makegymenv(stock_name, start_date, num_days, interval, normalize=False, init_balance=init_balance)


[*********************100%***********************]  1 of 1 completed


/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)
/media/victoru/B612CEC512CE8A37/stonkdecisiondocker/stonkdecision/src/curatedataset.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data['momentum_stoch_rsi'].iloc[:10] = 0.5
/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


init env with max step:  1006
[*********************100%***********************]  1 of 1 completed
init env with max step:  1006


/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)
/media/victoru/B612CEC512CE8A37/stonkdecisiondocker/stonkdecision/src/curatedataset.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data['momentum_stoch_rsi'].iloc[:10] = 0.5
/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [3]:
# get trained stable agents' path inside trained_stable_agents folder
path = os.path.join(os.getcwd(), 'trained_stable_agents')
# get agent type without .zip
stable_agents_type = [re.sub('.zip', '', agent) for agent in os.listdir(path) if agent.endswith('.zip')]

In [4]:
# create stable agent objects
agents_list = []
for agent_type in stable_agents_type:
    # get the agent's path
    agent_path = os.path.join(path, agent_type+'.zip')
    agent = Agent(norm_env, agent_type, agent_path)
    agents_list.append(agent)

In [5]:
agents_list[0]

In [6]:
data = run_env(agents_list[0], env, 1, normalize=True)

Terminated:  False ; Truncated:  True
env current step  1006  env max step  1006
Episode:  0 Timestep: 1005  done


In [7]:
data

{'data': [{'state': [[20.799999237060547,
     21.474000930786133,
     20.733333587646484,
     21.368667602539062,
     0.7155718207359314,
     0.0,
     100.0,
     0.5,
     20000.0,
     20000.0,
     0.0,
     0.0,
     0.0,
     0.0],
    [21.399999618530273,
     21.683332443237305,
     21.036666870117188,
     21.149999618530273,
     0.020021161064505577,
     -0.017443599179387093,
     0.0,
     0.5,
     20000.0,
     20000.0,
     0.0,
     0.0,
     0.0,
     0.0],
    [20.857999801635742,
     21.23666763305664,
     20.3786678314209,
     20.974666595458984,
     0.21517035365104675,
     -0.044898129999637604,
     0.0,
     0.5,
     20000.0,
     20000.0,
     0.0,
     0.0,
     0.0,
     0.0],
    [21.107999801635742,
     21.14933204650879,
     20.799999237060547,
     21.10533332824707,
     0.31968358159065247,
     -0.05547287315130234,
     27.108081817626953,
     0.5,
     20000.0,
     20000.0,
     0.0,
     0.0,
     0.0,
     0.0],
    [21.0666675567

In [3]:
agenttype = 'random'
algotype = 'momentum_stoch_rsi'
# set the trading range as % of balance
trade_range = [0.2, 0.6]
# find the momentum_stoch_rsi position in obs_features
for i in range(len(obs_features)):
    if obs_features[i] == algotype:
        algotype_index = i
        break

tradalgo = TradingAlgorithm(algotype, algotype_index, trade_range)
algoagent = Agent(env, agenttype, algo = tradalgo)

In [4]:
# run the environment
num_episodes = 1
run_env(algoagent, env, num_episodes, normalize = False)

step check truncated:  True
step check max step:  1006
step check current step:  1006
Terminated:  False ; Truncated:  True
env current step  1006  env max step  1006
Episode:  0 Timestep: 1006  done


{'data': [{'state': [[20.799999237060547,
     21.474000930786133,
     20.733333587646484,
     21.368667602539062,
     0.7155718207359314,
     0.0,
     100.0,
     0.5,
     20000.0,
     20000.0,
     0.0,
     0.0,
     0.0,
     0.0],
    [21.399999618530273,
     21.683332443237305,
     21.036666870117188,
     21.149999618530273,
     0.0200211617729182,
     -0.017443599863948833,
     0.0,
     0.5,
     20000.0,
     20000.0,
     0.0,
     0.0,
     0.0,
     0.0],
    [20.857999801635742,
     21.23666763305664,
     20.3786678314209,
     20.974666595458984,
     0.2151703549638058,
     -0.04489813037427837,
     0.0,
     0.5,
     20000.0,
     20000.0,
     0.0,
     0.0,
     0.0,
     0.0],
    [21.107999801635742,
     21.14933204650879,
     20.799999237060547,
     21.10533332824707,
     0.31968357680954074,
     -0.05547287353889274,
     27.108081491253245,
     0.5,
     20000.0,
     20000.0,
     0.0,
     0.0,
     0.0,
     0.0],
    [21.06666755676269